##### DB 연결

In [1]:
# DB Connection

import configparser
import pymysql

# config open
config = configparser.ConfigParser()
config.read('../config/config.ini')

class DBConnection:
    def __init__(self):
        self._conn = pymysql.connect(
            # 운영 DB
            # host = config['DB_ESG']['HOST'],
            # port = 3306,
            # user = config['DB_ESG']['USER'],
            # password = config['DB_ESG']['PASSWD'],
            # db = config['DB_ESG']['DB_NAME'],

            # Local DB
            host = config['DB_TEST']['HOST'],
            port = 3306,
            user = config['DB_TEST']['USER'],
            password = config['DB_TEST']['PASSWD'],
            db = config['DB_TEST']['DB_NAME'],

            #autocommit = True,           
            charset = 'utf8mb4',
            cursorclass = pymysql.cursors.DictCursor # DB조회시 컬럼명을 동시에 보여줌
        )
        self._cursor = self._conn.cursor()
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

    @property
    def connection(self):
        return self._conn
    
    @property
    def cursor(self):
        return self._cursor
    
    def rollback(self, rollback=True):
        if rollback:
            self.rollback()
        self.connection.close()
    
    def commit(self):
        self.connection.commit()
    
    def close(self, commit=True):
        if commit:
            self.commit()
        self.connection.close()
    
    def execute(self, sql, params=None):
        self.cursor.execute(sql, params or ())

    def executemany(self, sql, params=None):
        self.cursor.executemany(sql, params or ())

    def fetchall(self):
        return self.cursor.fetchall()
    
    def fetchone(self):
        return self.cursor.fetchone()
    
    def query(self, sql, params=None):
        self.cursor.execute(sql, params or ())
        return self.fetchall()
    
    def rows(self):
        return self.cursor.rowcount    

##### SQL class 함수

In [7]:
dbconn = DBConnection()

class DBClass:
    def __init__(self):
        pass
    
    def select_one(self, sql, args=None):
        dbconn.execute(sql, args)
        result = dbconn.fetchone()
        return result
    
    def select_all(self, sql, args=None):
        dbconn.execute(sql, args)
        result = dbconn.fetchall()
        return result
    
    def save_all(self, sql, args=None):        
        try:
            dbconn.executemany(sql, args)
            dbconn.commit() # commit()
            
            return 1
        except:
            dbconn.rollback() # rollback() & close()
            return 0
    
    def save_all_close(self, sql, args=None):
        try:
            dbconn.executemany(sql, args)
            dbconn.commit()
            dbconn.close() # commit() & close()
            return 1
        except:
            dbconn.rollback()
            return 0
    
    def db_execute(self, sql, args=None):
        dbconn.execute(sql, args)
        dbconn.commit()
            
    def db_commit(self):
        dbconn.commit()

    def db_close(self):
        dbconn.close()

##### 카테고리(tb_category) 데이터 입력

In [8]:
import pandas as pd

path = '../assets/'
file_name = 'keyword_list.csv'

file_path = path + file_name

df = pd.read_csv(file_path, encoding='utf-8')

# 중복 제거
cate = df.drop_duplicates(['카테고리'])
args = list(cate['카테고리'].values)

# print(args)

db_exec = DBClass()

str_query = """
    INSERT INTO tb_category(cate_name)VALUES(%s);
"""

for arg in args:
    db_exec.db_execute(str_query, arg)

db_exec.db_close()